In [1]:
import numpy as np
import random
import time
import os

In [2]:
with open('words.txt') as f:
    words = f.read()
words = words.split("\n")
words = [w for w in words if len(w) == 5]

In [3]:
def possible(w, state, memo):
    """
    inputs:
        w: the word to check if it's possible given a state
        state: the clue and guesses
                        each clue is a tuple of (l, c, i),
                        the letter, clue and index
                        and guesses is a set of LETTERS guessed
    
    outputs:
       True if w is consistent with state else False 
    """
    if (w, state) in memo: return memo[(w, state)]
    colors, letters, guesses = state

    # must have a yellow letter but not at yellow index
    for i,c in enumerate(colors):
        # must have green letter in same position
        if c == 'g' and not w[i] == letters[i]:
            memo[(w, state)] = False
            return False
        
        # check if yellow letter at the yellow idx
        if c == 'y':
            if w[i] == letters[i]:
                memo[(w, state)] = False
                return False 
        
            # now know yellow idx does not have the yellow letter, so check
            # if it is there
            if letters[i] not in w:#w.find(letters[i]) == -1:
                memo[(w, state)] = False
                return False 

    # black letters are in guesses, but not a yellow letter or green letter
    black_letters = [l   for l in guesses if  not l in letters]
    # no blacklist letter
    for i in range(5):
        if w[i] in black_letters: 
            memo[(w, state)] = False
            return False

    memo[(w, state)] = True
    return True
    
def possible_v(words, state, memo):
    """
    inputs:
        words [list of str]: the words to check if it's possible given a state
        state [tuples]: the clue and guesses
                        each clue is a tuple of (l, c, i),
                        the letter, clue and index
    
    outputs:
       True if w is consistent with state else False 
    """
    if state in memo: return memo[state]
    clues, guesses = state
    words = np.array([list(w) for w in words])
    sz = len(words)
    
    res = np.array([True]*sz)
    
    # must have green letter in same position
    green_letters = tuple(c for c in clues if c[1] == 'g')
    for g in green_letters:
        wh = np.where(words[:, g[2]] != g[0])
        res[wh] = False
        
    # must have a yellow letter but not at yellow index
    yellow_letters = tuple(c for c in clues if c[1] == 'y')
    for y in yellow_letters:
        # check if yellow letter at the yellow idx
        wh = np.where(words[:, y[2]] == y[0])
        res[wh] = False 
        # now know yellow idx does not have the yellow letter, so check
        # if it is there
        # where(res != False && no y[0] across the matrix)
        wh = np.where(  (res != False) 
                      & (np.array([y[0] in word for word in words]) == False) )
        res[wh] = False
            
    # black letters are in guesses, but not a yellow letter or green letter
    black_letters = [l   for     l in ''.join([g for g in guesses])
                              if  not l in [y[0] for y in yellow_letters]
                              and not l in [g[0] for g in green_letters ]]
    
    #for l in black_letters:
    #    wh = np.where( np.array([l in word for word in words]) == True)
    #    res[wh] = False   
    for j,w in enumerate(words):
        for i,l in enumerate(w):
            if l in black_letters:
                res[j] = False
    memo[state] = res
    return res


def solve(state, possible_words):
    """
    inputs:
        state [tuples]: the clue and guesses
                        each clue is a tuple of (l, c, i),
                        the letter, clue and index
        possible_words [list]: the 'dictionary' of 5 letters words
    
    outputs:
       best guess(es) given the state and words, and the avg number of words it eliminates
       
    state: ( ((l0,c0,i0), (l1,c1,i1), (l2,c2,i2), (l3,c3,i3), (l4,c4,i4)), 
             (guess1, guess2, ... ) )
           where l is the letter a-z or @ if unknown
           and c is the color black, yellow, green,
           and i is the index of letter
               black: unknown letter 
               yellow: correct letter, incorrect position
                       *note a correct word does NOT have 
                        this in the position  
               green: correct letter, correct position
           and guess1, guess2, ..., are the previous guesses
    """
    colors, letters, guesses = state
    memo = {}
    # trim the possible words given the current state
    i = 0
    while i < len(possible_words):
        if not possible(possible_words[i], state, memo): possible_words.pop(i)
        else: i += 1
    
    # assign a score to each word based on how many it eliminates
    # the guess will be the one that eliminates the most
    # word --> avg number would be eliminated
    avg_eliminated = {}

    def colors_and_letters(guess, answer):
        colors, letters = ['b']*5, ['@']*5
        for idx in range(5):
            if guess[idx] == answer[idx]:
                colors[idx], letters[idx] = 'g', guess[idx]
            elif answer.find(guess[idx]) != -1: 
                colors[idx], letters[idx] = 'y', guess[idx]
                
        return ''.join(colors), ''.join(letters)
    """
    def color_and_letter(idx, guess, answer):
        #res = f'{ f'{guess[idx]}g' if guess[idx] == answer[idx] else '@b'}{idx}'
        #if guess[idx] == answer[idx]: res[0] = [guess[idx], 'g']
        res = ['@', 'b']
        if guess[idx] == answer[idx]: res = [guess[idx], 'g']
        if res[1] != 'g' and answer.find(guess[idx]) != -1: res = [guess[idx], 'y']
        return tuple(res)
    """
    for i,guess in enumerate(possible_words):
        start = time.time()
        eliminated = 0
        for ans in possible_words:
            # clue: 'ccccc', '@@@@@',
            # 1) compute the new clue, which along with guess create state
            #wouldbe_state = (
            #                 tuple(color_and_letter(k, guess, ans) + (k,) for k in range(5)), # clue 
            #                 ''.join([guess, ''.join([g for g in guesses])]) # guesses
            #                )
            wouldbe_state = colors_and_letters(guess, ans) + (''.join([guess, ''.join([g for g in guesses])]),)
            #print(wouldbe_state)
            #eliminated += sum(~possible_v(possible_words, wouldbe_state, memo))
            
            eliminated += sum([not possible(w, wouldbe_state,memo) 
                               for w in possible_words])
            #for w in possible_words:
                # 2) remove any word inconsistent with the clue
                #eliminated += int(not possible(w, wouldbe_state, memo))
            
        #print(f'Completed {i}/{len(possible_words)} ({time.time() - start} sec)')
        avg_eliminated[guess] = eliminated / len(possible_words)
        
    # now the map from guess --> number of words eliminated
    # we we'll return the one that maximize the number of eliminated words
    # find the max number of words (on avg) eliminated
    m = max(avg_eliminated.items(), key=lambda item: item[1])[1]
    
    # best guess is one(s) that eliminates the most 
    best_guesses = [item[0] for item in avg_eliminated.items() if item[1] == m ]
    
    return best_guesses, m

In [4]:
i, j = 2293, 2593
best_guesses, eliminated = solve(
    ("bbbbb", "@@@@@", tuple(),), words[i : j + 1]
)
print("Best guess(es) out of word [{}, {}]: {}".format(i, j, best_guesses))
print("Average eliminated: {}".format(eliminated))

Best guess(es) out of word [2293, 2593]: ['soare']
Average eliminated: 293.1827242524917
